In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
!cd /content/drive/'path'

/bin/bash: line 1: cd: /content/drive/path: No such file or directory


In [1]:
!pip install faster-whisper
!pip install gradio
!pip install googletrans==4.0.0-rc1
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 11.7 MB/s et

In [5]:
from faster_whisper import WhisperModel

stt_model = WhisperModel("large-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

In [6]:
# stt를 실행하는 함수
def stt_func(filepath):

  transcripts = []

  segments, info = stt_model.transcribe(filepath, language="ko")
  tmp = ""
  for segment in segments:
    tmp = tmp + segment.text
    transcripts.append(tmp)

  return transcripts

In [7]:
record = []
def trans_audio(filepath):
    segments, info = stt_model.transcribe(filepath, language="en", task = "transcribe")
    tmp = ""
    for segment in segments:
        tmp = tmp + segment.text
        record.append(tmp)
    return tmp

import gradio as gr
import time

def recording():
  output_1 = gr.Textbox(label="Speech to Text")

  gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI',
    fn = trans_audio,
    inputs=[gr.Audio(type="filepath")],
    outputs = [output_1],
    live=True).launch()

In [8]:
from googletrans import Translator
def translate_file(input_file_path, output_file_path, src_lang='ko', dest_lang='en'):
    # 번역기 초기화
    translator = Translator()

    # 입력 파일 읽기
    with open(input_file_path, 'r', encoding='utf-8') as file:
        korean_text = file.read()

    # 한국어 텍스트를 영어로 번역
    translation = translator.translate(korean_text, src=src_lang, dest=dest_lang)
    english_text = translation.text

    # 번역된 텍스트를 출력 파일로 저장
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(english_text)

In [38]:
import pandas as pd

# Google Drive에 저장된 CSV 파일 경로
file_path = '/content/drive/MyDrive/MBTI 500.csv'

# CSV 파일 로드
df = pd.read_csv(file_path)

# 기존에 T 또는 F를 포함하는 성격 유형에 대해 해당 문자로 대체
df['type'] = df['type'].replace({
    'INTJ': 'T', 'INTP': 'T', 'ENTJ': 'T', 'ENTP': 'T',
    'ISTJ': 'T', 'ISTP': 'T', 'ESTJ': 'T', 'ESTP': 'T',
    'INFJ': 'F', 'INFP': 'F', 'ENFJ': 'F', 'ENFP': 'F',
    'ISFJ': 'F', 'ISFP': 'F', 'ESFJ': 'F', 'ESFP': 'F'
})

# 데이터 프레임 확인
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,T
1,rap music ehh opp yeah know valid well know fa...,T
2,preferably p hd low except wew lad video p min...,T
3,drink like wish could drink red wine give head...,T
4,space program ah bad deal meing freelance max ...,T


In [44]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 텍스트 데이터와 라벨 추출
texts = df['posts'].values
labels = df['type'].values

T_cnt = 0
F_cnt = 0
x_cnt = 0
for i in labels:
  if i == 'T':
    T_cnt += 1
  elif i == 'F':
    F_cnt += 1
  else :
    x_cnt += 1

print(T_cnt)
print(F_cnt)
print(x_cnt)

# 텍스트 토큰화
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 패딩 적용
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

# 라벨 인코딩 (T -> 0, F -> 1)
label_dict = {'T': 0, 'F': 1}
labels = np.array([label_dict[label] for label in labels])

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


69203
36864
0


In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# LSTM 모델 구축
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2122/2122 ━━━━━━━━━━━━━━━━━━━━ 255s 119ms/step - accuracy: 0.6885 - loss: 0.5978 - val_accuracy: 0.7206 - val_loss: 0.5366
Epoch 2/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 258s 121ms/step - accuracy: 0.7577 - loss: 0.5046 - val_accuracy: 0.8008 - val_loss: 0.4423
Epoch 3/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 254s 120ms/step - accuracy: 0.8124 - loss: 0.4218 - val_accuracy: 0.8010 - val_loss: 0.4396
Epoch 4/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 255s 120ms/step - accuracy: 0.8289 - loss: 0.3920 - val_accuracy: 0.8000 - val_loss: 0.4419
Epoch 5/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 256s 121ms/step - accuracy: 0.8421 - loss: 0.3684 - val_accuracy: 0.7982 - val_loss: 0.4552


In [46]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# 예측 함수
def predict_mbti(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=maxlen)
    prediction = model.predict(padded_sequence)
    print(f'predic: {prediction}')
    return 'T' if prediction < 0.5 else 'F'

# 예측 예시
sample_text = "I'm so sad"
predicted_type = predict_mbti(sample_text)
print(f'Predicted MBTI type: {predicted_type}')


663/663 ━━━━━━━━━━━━━━━━━━━━ 22s 33ms/step - accuracy: 0.7947 - loss: 0.4677
Test Accuracy: 0.7900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
predic: [[0.2566249]]
Predicted MBTI type: T


In [47]:
recording()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://64219e71f286312a00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [48]:
record = []
record.append('내가 오늘은 기분이 좋아')
record.append('한국의 술은 쌀로 만들어집니다')
record.append('내일 만나서 얘기하자')
record.append('그랬구나, 많이 속상했구나')
record.append('얼어붙은 한강 위로 고양이가 걸어다닙니다')


for i in record:
  print((i))
  k = Translator().translate(i, src='ko', dest='en').text
  print(k)
  predicted_type = predict_mbti(k)
  print(f'Predicted MBTI type: {predicted_type}')
  print()



내가 오늘은 기분이 좋아
I feel good today
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
predic: [[0.13043444]]
Predicted MBTI type: T

한국의 술은 쌀로 만들어집니다
Korean liquor is made of rice
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
predic: [[0.19261695]]
Predicted MBTI type: T

내일 만나서 얘기하자
Let's meet tomorrow and talk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
predic: [[0.15366437]]
Predicted MBTI type: T

그랬구나, 많이 속상했구나
I did, you were a lot upset
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
predic: [[0.16375822]]
Predicted MBTI type: T

얼어붙은 한강 위로 고양이가 걸어다닙니다
The cat walks over the frozen Han River
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
predic: [[0.09028429]]
Predicted MBTI type: T

